In [1]:
import node
import numpy as np

In [2]:
class Activate(node.Op):

    def __init__(self, x, *args):
        super().__init__(x)
        
        # フォワード時はサイン関数と同じ
        self.output = np.sign(x.value)

    def backward(self, err_sig):
        # バックワード時はハードシグモイド関数と同じ
        return self._srcs[0].acc_grad(err_sig * 1 * (np.abs(self.output) <= 1))

In [3]:
from node.node import _single_oprand_op

@_single_oprand_op
def activate(self):
    return Activate(self)

setattr(node.Node, "activate", activate)

In [4]:
class BinaryLinear(node.Layer):
    
    def __init__(self, num_in_units, num_out_units):
        super(BinaryLinear, self).__init__()
        
        # ウェイトだけ使う
        self.parameters = {"W": node.Node(np.random.randn(num_in_units, num_out_units))}
        
    def __call__(self, input):
        return input.dot(self.parameters["W"].activate())

In [5]:
class BinaryNet(node.Network):
    
    def __init__(self, num_in_units, num_h_units, num_out_units):
        self.layers = [BinaryLinear(num_in_units, num_h_units),
                       node.BatchNorm(num_h_units),
                       BinaryLinear(num_h_units, num_h_units),
                       node.BatchNorm(num_h_units),
                       BinaryLinear(num_h_units, num_out_units),
                       node.BatchNorm(num_out_units)]
        
    def __call__(self, input):
        hidden = input
        hidden = self.layers[1](self.layers[0](hidden)).activate()
        hidden = self.layers[3](self.layers[2](hidden)).activate()
        hidden = self.layers[5](self.layers[4](hidden))
        return hidden
    
classifier = BinaryNet(784, 1024, 10)
optimizer = node.Adam(classifier.get_parameters(), 0.001)

In [6]:
train_dataset = node.MNIST(training=True)
train_dataloader = node.DataLoader(train_dataset, batch_size=100)

In [7]:
test_dataset = node.MNIST(training=False)
test_dataloader = node.DataLoader(test_dataset, batch_size=100)

In [8]:
def train(input, target):
    optimizer.zero_grad()
    
    #　パラメーターを更新する
    output = classifier(input/255).softmax_with_cross_entropy(target)
    output.backward()
    optimizer()
    
    return output.value

In [9]:
def measure(prediction, target):
    # 出力とラベルを受け取り、何個正解したかの数を返す。
    
    prediction = np.argmax(prediction.value, axis=1)
    target = np.argmax(target.value, axis=1)
    
    return np.sum(np.where(prediction == target, 1, 0))

In [10]:
def evaluate(input, target):
    with node.zero_grad():
        prediction = classifier(input/255)
        output = prediction.softmax_with_cross_entropy(target)
        
    loss = output.value
    accuracy = measure(prediction, target)
    
    return loss, accuracy

In [11]:
for epoch in range(501):
    
    # Train Loss, Test Loss, Accuracy
    metrics = [0, 0, 0]
    
    classifier.training()
    for input, target in train_dataloader:
        metrics[0] += train(input, target)

    classifier.evaluation()
    for input, target in test_dataloader:
        loss, accuracy = evaluate(input, target)
        metrics[1] += loss 
        metrics[2] += accuracy
        
    metrics[0] /= len(train_dataloader)
    metrics[1] /= len(test_dataloader)
    metrics[2] /= 100 * len(test_dataloader)
    
    if epoch % 100 == 0:
        print("epoch {0:3}, training loss {1:.4f}, test loss {2:.4f}, accuracy {3:.2f}".format(epoch, *metrics))

epoch   0, training loss 2.3199, test loss 1.2416, accuracy 0.78
epoch 100, training loss 0.4515, test loss 0.4103, accuracy 0.93
epoch 200, training loss 0.3986, test loss 0.3862, accuracy 0.93
epoch 300, training loss 0.3624, test loss 0.3786, accuracy 0.94
epoch 400, training loss 0.3470, test loss 0.3777, accuracy 0.93
epoch 500, training loss 0.3455, test loss 0.3810, accuracy 0.93
